In [ ]:
# !pip install -q efficientnet

In [ ]:
import math, re, os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import tensorflow as tf
import tensorflow.keras.layers as L
# from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0, EfficientNetV2B3, EfficientNetV2S
# from tf.keras.applications.efficientnet import EfficientNetB3
from sklearn import metrics
print("hi")


In [ ]:
print("hi")

tf.random.set_seed(42)

In [ ]:
with open('/kaggle/working/hi.txt', 'w') as file:
    file.write("hi")
    file.close()

In [ ]:
!ls -l /kaggle/working/efficientnetv2_s.train-n-240000.weights.01-0.7304.h5

# TPU Strategy and other configs 

In [ ]:
import tensorflow as tf
print('hi')

# Detect and initialize TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()  # Default strategy for CPU and single GPU

print('Number of replicas:', strategy.num_replicas_in_sync)

In [ ]:
# !rm -r /kaggle/input/alaska2-image-steganalysis/Test
# !rm /kaggle/input/alaska2-image-steganalysis/sample_submission.csv

# Setup the Tensorflow datasets (train and val), data augmentations, and oversampling if necessary

In [ ]:
import tensorflow as tf
import os

# Constants
base_dir = '/kaggle/input/alaska2-image-steganalysis/'
class_names = ['Cover', 'JMiPOD', 'UERD', 'JUNIWARD']
# class_names = ['Cover', 'JMiPOD']

image_size = (512, 512)
# 20 for effnet-M, 32 for effnet-S
batch_size = 32 * strategy.num_replicas_in_sync
augmented_batch_size = 4
        
def get_paths(split_at, end_at, oversample=True,multiclass=False):
    
    train_class_paths_array = []
    val_class_paths_array = []
    
    for class_name in class_names:
        class_dir = os.path.join(base_dir, class_name)
        all_images = os.listdir(class_dir) 

        selected_images = all_images[:end_at]  
        
        train_class_paths = []
        val_class_paths = []
            
        for img in selected_images[:split_at]:
            train_class_paths.append(os.path.join(class_dir, img))           
                
        for img in selected_images[split_at:]:
            val_class_paths.append(os.path.join(class_dir, img))            
        
        train_class_paths_array.append(train_class_paths)
        val_class_paths_array.append(val_class_paths)
#     if multiclass:
#             val_labels.append(tf.one_hot(class_names.index(class_name), len(class_names)))
    
    train_data_paths = []
    val_data_paths = []
    
    # 'Cover' paths must be first in array
    for i, cover_path in enumerate(train_class_paths_array[0]):
        if not oversample:
            train_data_paths.append(cover_path)
        for j in range(1, 4): # 1, 2 ,3
            if oversample:
                train_data_paths.append(cover_path)
            train_data_paths.append(train_class_paths_array[j][i])
        
    for i, cover_path in enumerate(val_class_paths_array[0]):
        if not oversample:
            val_data_paths.append(cover_path)
        for j in range(1, 4): # 1, 2 ,3
            if oversample:
                val_data_paths.append(cover_path)
            val_data_paths.append(val_class_paths_array[j][i])
    
    train_labels = [0 if 'Cover' in path else 1 for path in train_data_paths]
    val_labels = [0 if 'Cover' in path else 1 for path in val_data_paths]
    
    return train_data_paths, val_data_paths, train_labels, val_labels

# 80/20 split
# split_at = 25600
# end_at = 32000
split_at = 60000
end_at = 75000

train_data_paths, val_data_paths, train_labels, val_labels = get_paths(split_at, end_at, oversample=True)

print(train_data_paths[:10])
print(val_data_paths[:10])

train_labels = tf.cast(train_labels, tf.float32)
val_labels = tf.cast(val_labels, tf.float32)

train_len = len(train_data_paths)
val_len = len(val_data_paths)

preprocessor = tf.keras.applications.resnet50.preprocess_input
preprocess = False # If preprocessing is needed, EfficientNetV2 has a preprocessing layer doesn't need it
augment = True    

# Function to load and preprocess an image
def load_and_preprocess_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    if preprocess:
        img = preprocessor(img)  # Resnet preprocessing
        img = img / 255.0
    if augment:
        img = augment_image(img)
    return img

# Function to apply data augmentation to an image
def augment_image(img):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    # Add more augmentation techniques as needed
    return img

# # Function to create a batch of augmented images
# def create_augmented_batch(img):
#     augmented_images = [augment_image(img) for _ in range(augmented_batch_size)]  # Create 4 augmented images
#     return tf.stack(augmented_images)

# for x, y in train_data_paths:
#     print(x, y)
#     load_and_preprocess_image(x, y)

# Create datasets inside strategy.scope()
with strategy.scope():
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((train_data_paths, train_labels))
#         .shuffle(buffer_size=len(train_labels), seed=42)
        .map(lambda img_path, label: (load_and_preprocess_image(img_path), label))
        .batch(batch_size)
        .repeat()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )
    print('train dataset')
    
    val_dataset = (
        tf.data.Dataset
        .from_tensor_slices((val_data_paths, val_labels))
#         .shuffle(buffer_size=len(val_labels), seed=42)
        .map(lambda img_path, label: (load_and_preprocess_image(img_path), label))
        .batch(batch_size)
        .repeat()
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )
#     val_dataset = tf.data.Dataset.from_tensor_slices((val_data_paths, val_labels))
#     val_dataset = val_dataset.map(lambda img_path, label: (load_and_preprocess_image(img_path), label))
#     val_dataset = val_dataset.shuffle((end_at - split_at)).repeat().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    print('val datasset')
    
    # Check the dataset shapes
    for image, label in train_dataset.take(1):
        print("Training Batch - Image Shape:", image.shape, "Label:", label)
        print(tf.math.reduce_max(image))
        print(tf.math.reduce_min(image))        
    for image, label in val_dataset.take(1):
        print("Validation Batch - Image Shape:", image.shape, "Label:", label)
        print(tf.math.reduce_max(image))
        print(tf.math.reduce_min(image)) 
    print("Train len: ", train_len)
    print("Val len: ", val_len)


In [ ]:
# import os
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Define the base directory containing the 'Cover', 'JMiPOD', 'UERD', and 'JUNIWARD' folders
# base_dir = '/kaggle/input/alaska2-image-steganalysis'

# # Specify image size and batch size
# img_size = (512, 512)
# batch_size = 32 * strategy.num_replicas_in_sync

# # Configuration
# # epochs = 20

# # Define data augmentation parameters
# datagen = ImageDataGenerator(
# #     rescale=1./255,
#     horizontal_flip=True,
#     vertical_flip=True,
#     validation_split=0.2  # 20% of the data will be used for validation
# )

# with strategy.scope():

#     print("starting train gen")
#     # Create a generator for training data
#     train_generator = datagen.flow_from_directory(
#         base_dir,
#         target_size=img_size,
#         batch_size=batch_size,
#         classes=['Cover', 'JMiPOD', 'JUNIWARD', 'UERD'],
#         class_mode='categorical',
#         subset='training',  # Specify that this is the training subset
#     )
    
#     # Create a generator for validation data
#     validation_generator = datagen.flow_from_directory(
#         base_dir,
#         target_size=img_size,
#         batch_size=batch_size,
#         classes=['Cover', 'JMiPOD', 'JUNIWARD', 'UERD'],
#         class_mode='categorical',
#         subset='validation',  # Specify that this is the validation subset
#     )


In [ ]:
# def generate_dummy_image(shape=(512, 512, 3)):
#     return np.random.rand(*shape).astype('float32') / 255.0

# def generate_dummy_label(num_classes=10):
#     return np.random.randint(num_classes, size=1)[0]

# def create_dummy_dataset(num_samples, num_classes=10, batch_size=32):
#     images = [generate_dummy_image() for _ in range(num_samples)]
#     labels = [generate_dummy_label(num_classes) for _ in range(num_samples)]
#     labels_one_hot = [tf.one_hot(label, num_classes) for label in labels]
    
#     dataset = tf.data.Dataset.from_tensor_slices((images, labels_one_hot))
#     dataset = dataset.shuffle(buffer_size=num_samples)
#     dataset = dataset.batch(batch_size)
    
#     return dataset

# # Example usage:
# train_samples = 128
# validation_samples = 64
# num_classes = 4
# batch_size = 32

# with strategy.scope():

#     train_generator = create_dummy_dataset(train_samples, num_classes, batch_size)
#     validation_generator = create_dummy_dataset(validation_samples, num_classes, batch_size)

# Define and Compile Model, Loss function, Optimizer, Metrics

In [ ]:
# base_model = EfficientNetV2(weights='imagenet', include_top=False, input_shape=(512, 512, 3))
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0, EfficientNetV2B3, EfficientNetV2S, EfficientNetV2M, EfficientNetV2L
from tensorflow.keras.applications.resnet50 import ResNet50

with strategy.scope():

#     tf.keras.applications.efficientnet_v2.EfficientNetV2B3(
#         include_top=False,
#         weights='imagenet',
#         input_tensor=None,
#         input_shape=None,
#         pooling=None,
#     #     classes=1000,
#         classifier_activation='softmax',
# #         include_preprocessing=True # should rescale for us
#     )
    
    # Image preprocessing (normalization) included in implementation
    # https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet/preprocess_input 
    base_model = EfficientNetV2S(
        include_top=False,
        weights=None,
#         input_tensor=None,
        input_shape=(512,512, 3),
#         pooling=None,
#         classes=num_classes,
#         classifier_activation='softmax',
        include_preprocessing=True 
    )
    
#     base_model = EfficientNetV2M(
#         include_top=False,
#         weights=None,
#         input_shape=(512,512, 3),
#         include_preprocessing=True 
#     )
    
#     base_model = ResNet50(
#         include_top=False,
#         weights=None,
# #         input_tensor=None,
#         input_shape=(512, 512, 3),
# #         pooling='avg',
# #         classes=4,
#     )

    # If we want to fine-tune
#     for layer in base_model.layers:
#         layer.trainable = False   
#     model_name = 'resnet50'
    model_name = 'efficientnetv2_s'
    pretrained = False    
    num_classes = 4
    
    model = tf.keras.Sequential([
        base_model,
        L.GlobalAveragePooling2D(),
        L.Dropout(0.2),
        L.Dense(1, activation='sigmoid') # One output for binary classification
#         tf.keras.layers.Dense(num_classes, activation='softmax')
#         SimpleAttention(),
#         L.Flatten(),
#         L.Dense(units=256), # one for each pixel (16 x 16)
#         L.Dropout(0.5),
#         L.Dense(num_classes, activation='softmax') 
    ])


    print(model.summary())

    model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score()]) # try f1-score
#     model.compile(optimizer='adam',
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

# Custom Callback to save a running history of important metrics each epoch

In [ ]:
import pickle
from tensorflow.keras.callbacks import Callback

class SaveHistoryCallback(Callback):
    def __init__(self, filename=f'/kaggle/working/no_shuffle_{model_name}' + ('.pretrained' if pretrained else '') + f'.train-n-{train_len}.running_history.pkl'):
        super(SaveHistoryCallback, self).__init__()
        self.filename = filename
        self.history = {
            "loss": [],
            "accuracy": [],
            "val_loss": [],
            "val_accuracy": [],
            "precision": [],
            "recall": [],
            "f1_score": []
        }
    
    def on_epoch_end(self, epoch, logs=None):
#         current_epoch_data = {
#             'epoch': epoch,
#             'logs': logs.copy() if logs else None
#         }        
        if logs is not None:
            # Access training metrics for the current epoch
            loss = logs.get('loss')
            accuracy = logs.get('accuracy')
            
            # Access validation metrics if available
            val_loss = logs.get('val_loss')
            val_accuracy = logs.get('val_accuracy')

            self.history["loss"].append(loss)
            self.history["accuracy"].append(accuracy)
            self.history["val_loss"].append(val_loss)
            self.history["val_accuracy"].append(val_accuracy)
            
            precision = logs.get('precision')
            recall = logs.get('recall')
            f1_score = logs.get('f1_score')
            
            self.history["precision"].append(precision)
            self.history["recall"].append(recall)
            self.history["f1_score"].append(f1_score)

#             with open(self.filename, 'w') as file:
#                 json.dump(self.history, file)
#                 print(f'saved to {self.filename}')
            with open(self.filename, 'wb') as file:
                print(f'saved history to {self.filename}')
                pickle.dump(self.history, file)
        else:
            print('logs unexpectedly none in SaveHistoryCallback')

#             # Custom actions based on metrics
#             print(f' Epoch {epoch + 1}: Val Loss={val_loss:.4f}, Val Accuracy={val_accuracy:.4f}')


# Load a checkpoint if necessary

In [ ]:
load_checkpoint = True
# load_path = '/kaggle/working/efficientnetv2_s.train-n-240000.weights.01-0.7304.h5'
load_path = '/kaggle/input/binary-effnetv2/binary_no_shuffle_efficientnetv2_s.train-n-360000.weights.06-0.5020.h5'

if load_checkpoint:
    model.load_weights(load_path)
    print(f'loaded from {load_path}')

# Train the model (with checkpoints and scheduler)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

num_epochs = 7

with strategy.scope():
#     checkpoint = ModelCheckpoint('/kaggle/input/weights.{epoch:02d}-{val_loss:.2f}.h5', save_freq=1, monitor='val_loss', save_best_only=True)
    
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/binary_no_shuffle_' + model_name + ('.pretrained' if pretrained else '') + '.train-n-' + str(train_len) + '.weights.{epoch:02d}-{val_accuracy:.4f}.h5',
        save_weights_only=True,
        verbose=1,
        monitor='val_accuracy',
#         mode='max',
#         save_best_only=True
    )
    
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_f1_score',  
        factor=0.1,           # Factor by which the learning rate will be reduced (new_lr = lr * factor)
        patience=2,           # Number of epochs with no improvement after which learning rate will be reduced
        min_lr=1e-8,          # Minimum learning rate
        verbose=2             # 1 for progress bar, 2 for logs (when running in background)
    )
    
    multiclass = False
    
    # Calculate class weights based on the distribution of classes
#     class_weights = {
#         0: 1.0,  # weight for class 0 (e.g., cover images)
#         1: train_len / (3 * train_len),  # weight for class 1 (e.g., stego images)
#     } if multiclass is False else None
    
#     print(f'class_weights: {class_weights}')
    
    history = model.fit(
        train_dataset,
        steps_per_epoch=train_len // batch_size,
        epochs=num_epochs,        
        validation_data=val_dataset,
        validation_steps=val_len // batch_size, 
        callbacks=[checkpoint_callback, SaveHistoryCallback(), lr_scheduler],
#         verbose=2
#         class_weight=class_weights
    )
    
    # Save the training history to a file
#     import pickle

    with open(f'/kaggle/working/no_shuffle_{model_name}' + ('.pretrained' if pretrained else '') + f'.train-n-{train_len}.final_history.pkl', 'wb') as file:
        pickle.dump(history.history, file)

In [ ]:
# for images, labels in train_dataset.take(1):
#     print(model.predict(images))

In [ ]:
model.save(f'/kaggle/working/binary_no_shuffle_{model_name}.{'pretrained' if pretrained else ''}_model.keras')

# Evaluation

- Confusion Matrix + AUC Curve
- Accuracy + Loss Graphs

In [ ]:
# Loads the weights
load_weights = False
# load_path = '/kaggle/working/efficientnetv2_s.pretrained.train-n-32000.weights.04-0.7500.h5'
load_path = '/kaggle/input/binary-effnetv2/binary_no_shuffle_efficientnetv2_s.train-n-360000.weights.06-0.5020.h5'
# load_path = '/kaggle/working/efficientnetv2_s.train-n-32000.weights.01-0.7489.h5'

if load_weights:
    model.load_weights(load_path)
    print(f'loaded weights from {load_path}')

# y_true = np.empty()
# y_preds = np.empty()

from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report
import matplotlib.pyplot as plt

# Initialize empty lists for true labels and predicted probabilities
y_true = []
y_pred_prob = []

# Iterate over the validation dataset to get true labels and predicted probabilities
i = 0
print('evaluating to generate plots')
for images, labels in val_dataset:    
    y_true.append(labels.numpy()) 
    y_pred_prob.append(model.predict(images))
    
#     if i > 5:
#         break;
    
#     i += 1

# Concatenate the lists to obtain numpy arrays
y_true = np.concatenate(y_true, axis=0)
y_pred_prob = np.concatenate(y_pred_prob, axis=0)
print(y_pred_prob[:10])
# print(y_true)
# print(y_pred)

In [ ]:
# Convert predicted probabilities to binary predictions (0 or 1)
threshold = 0.50
y_pred = tf.cast(tf.math.greater_equal(y_pred_prob, threshold), dtype=tf.int32)

In [ ]:
# unique, idx, counts = tf.unique_with_counts(y_pred_prob.flatten())
# print(unique, counts)

In [ ]:
print(classification_report(y_true, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot Confusion Matrix
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['Cover', 'Stego']  
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Annotate each cell with its value
for i in range(len(classes)):
    for j in range(len(classes)):
        plt.text(j, i, str(conf_matrix[i, j]), ha='center', va='center', color='red')

plt.show()

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# model_name = 'resnet50'
# train_len = 100
# pretrained = True
# print('/kaggle/working/' + model_name + '.train-n-' + str(train_len) + '.weights.{epoch:02d}-{val_accuracy:.4f}.h5')
# print('/kaggle/working/' + model_name + ('.pretrained' if pretrained else '') + '.train-n-' + str(train_len) + '.weights.{epoch:02d}-{val_accuracy:.4f}.h5')
# print(f'/kaggle/working/{model_name}.train-n-{train_len}.final_history.pkl')
# print(f'/kaggle/working/{model_name}.train-n-{train_len}.running_history.json')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, ConfusionMatrixDisplay, confusion_matrix

# Accuracy and Loss Graphs
# history = model.fit(train_data, epochs=num_epochs, validation_data=validation_data)
import pickle

# num_epochs = 7

# running_history = True
# # running_history_path = f'/kaggle/working/{model_name}' + ('.pretrained' if pretrained else '') + f'.train-n-{train_len}.' + ('running_history.pkl' if running_history else 'final_history.pkl')
# running_history_path = '/kaggle/working/efficientnetv2_s.train-n-32000.running_history.pkl'
# # running_history_path = '/kaggle/working/resnet50.pretrained.train-n-32000.running_history.pkl'

# running_history_path = '/kaggle/working/efficientnetv2_s.train-n-240000.running_history.pkl'
# running_history_path = '/kaggle/input/binary-effnetv2/no_shuffle_efficientnetv2_s.train-n-360000.running_history.pkl'

# with open(running_history_path, 'rb') as file:
#     history = pickle.load(file)
# else:
history = history.history

print(history)


plt.figure()
plt.plot(range(1, num_epochs + 1), history['accuracy'], label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(range(1, num_epochs + 1), history['loss'], label='Training Loss')
plt.plot(range(1, num_epochs + 1), history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()